## Run this first if you want NDVI/NDWI basemaps

In [2]:
import ee
ee.Authenticate()


True

## Vibes

In [2]:
%load_ext autoreload
%autoreload 2

from src.ui import GeoVibes

vibes = GeoVibes(
    duckdb_directory = '/Users/christopherren/geovibes/local_databases',
    start_date = '2024-01-01',
    end_date = '2025-01-01',
    gcp_project='demeterlabs-gee', verbose=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Initializing GeoVibes...
✅ Earth Engine initialized with user credentials
  Found: /Users/christopherren/geovibes/local_databases/alabama_google.db
  Found: /Users/christopherren/geovibes/local_databases/bali_google.db
  Found: /Users/christopherren/geovibes/local_databases/java_vit_embeddings_20k.db
  Found: /Users/christopherren/geovibes/local_databases/java_google.db
Found 4 database(s) in /Users/christopherren/geovibes/local_databases
📁 Found 4 databases in directory
💾 Connecting to local database: /Users/christopherren/geovibes/local_databases/alabama_google.db
✅ Database connection established successfully
⚠️  Failed to create Earth Engine boundary: None: No such file or directory
⚠️  NDVI/NDWI basemaps will be unavailable
🗺️  spatial extension loaded for geometry support
🔍 Detected embedding dimension: 64
📍 Using boundary file: /Users/christopherren/geovibes/geometries/alabama.geojson
Buildin

HTML(value='\n        <style>\n        .widget-toggle-buttons button:nth-child(1).mod-active {\n            ba…

🗺️  Loading boundary layer: /Users/christopherren/geovibes/geometries/alabama.geojson
✅ Boundary layer added successfully


In [ ]:
Y
